# Create MODFLOW6-grid-based tiff files and model_grid.csv file from GIS data sources

In [1]:
# This first cell imports needed modules and sets a few display options

%matplotlib notebook

import os
import datetime as dt
import pickle, joblib

# Standard data science libraries
import pandas as pd
import numpy as np
import scipy.stats as ss
import scipy.optimize as so 
import scipy.interpolate as si

# Visualization
import matplotlib.pyplot as plt
# import seaborn as sns
plt.style.use('seaborn-notebook')

# Options for pandas
pd.options.display.max_columns = 20
pd.options.display.max_rows = 50

# Display all cell outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

from IPython.display import Image
from IPython.display import Math

import scipy.ndimage as nd
import scipy.spatial as sp
import geopandas as gpd
from shapely.geometry import Polygon
import gdal
gdal.UseExceptions()
import flopy as fp

import Genmod_Utilities as gmu
import json

In [2]:
root = 'C:/Users/jjstarn/Workspace/GIS for GSM--data processing'

# (src, attribute, name)

vector_list = [
  
    (os.path.join(root, 'K_from_Tashie', 'K_from_Tashie.shp'), 
     'Ksat_Wet', 'Ksat_Wet'),
    
    (os.path.join(root, 'K_from_Tashie', 'K_from_Tashie.shp'), 
     'Ksat_Dry', 'Ksat_Dry')
]


# (src, method, factor, name)

raster_list = [
    (os.path.join(root, 
                  'Shangguan_DTB_Predictions_Global', 
                  'BDTICM_M_250m_ll.tif'),
    gdal.GRA_Bilinear, 0.01, 'b_r_Shan'),

    (os.path.join(root, 
                  'Global_Soil_Regolith_Sediment_1304', 
                  'data',
                 'average_soil_and_sedimentary-deposit_thickness.tif'),
    gdal.GRA_Bilinear, 1., 'b_r_Pell'),
    
    (os.path.join(root, 
                  'T_from_Output_CONUS_trans_dtw, simulated depth to water and estimated transmissivity', 
                  'conus_MF6_SS_Unconfined_250_dtw.tif'),
    gdal.GRA_Bilinear, 1., 'b_u')
]

Read the National Groundwater Grid

In [3]:
ngwm_grid = os.path.join(root, 'T_from_Output_CONUS_trans_dtw, simulated depth to water and estimated transmissivity', 
                         'conus_MF6_SS_Unconfined_250_trans.tif')

natgrid = gmu.SourceProcessing(nodata=-99999.)
natgrid.read_raster(ngwm_grid)
proj = natgrid.output_raster_prj

natgrid.old_array[natgrid.old_array == natgrid.nodata] = np.nan

K_df = pd.DataFrame({'T': natgrid.old_array.ravel()})

In [6]:
for src, attribute, name in vector_list:
    natgrid.process_vector_data(src, attribute)
    K_df[name] = natgrid.new_array.ravel()

In [7]:
for src, method, factor, name in raster_list:
    natgrid.process_raster_data(src, method, factor)
    K_df[name] = natgrid.new_array.ravel() 

### Unit conversions

* GLHYMPS units are $100 * log_{10}$ k (intrinsic permeability, $m^2$).  To convert to hydraulic conductivity (K) to $m/d$ multiply by $10^{(k/100)}*10^7$ * 86400 $s/d$
* Tashie units are $cm/s$. To convert to $m/d$ multiply by 864
* Shangguan units are $cm$. The conversion factor of 0.01 was applied in the vector processing step.
* Zell and Sanford units are $m^2/d$ and do not need conversion
* Pelletier is in $m$

### Rationale for $K$ calculations.  

An effective transmissivity ($\tilde{T}$) for CONUS over an unknown thickness is given by Zell and Sanford. The spatial detail and resolution is really good and captures much of the known variation in $K$, for instance in areas of narrow riverine deposits in the northeast.  Their $\tilde{T}$ zones were based on Soller's mapping of surficial deposits of the CONUS. $\tilde{T}$ is dominated by the $K$ of surficial deposits where such deposits are thick and by the $K$ of bedrock elsewhere. $\tilde{T}$ was derived from model calibration to head data. The problem is to disaggregate the effective values of $\tilde{T}$ into $K$ and $b$ for surficial (subscript s) and bedrock (subscript b) units,  $\tilde{T} = {K_s}{b_s} + {K_b}{b_b}$. Rearranging and solving for $K_b$,

$K_b = \frac{\tilde{T}-{K_s}{b_s}}{b_b}$

Tashie et al. produced estimates of effective $\tilde{K}$ using a different approach. They used an analytical solution of the groundwater flow equation calibrated against streamflow recession data. 

$K_s \approx \tilde{K_s}$

$b_s \approx 3.7 m \qquad (dry)$

$b_s \approx 14 m \qquad (wet)$

$b_{r}$ is regolith thickness from Shangguan et al., 

$b_{sat\_r}$ is saturated zone thickness in regolith,

$b_{u}$ is unsaturated zone thickness (depth to water) from Zell and Sanford, and

$b_b$ is bedrock contributing zone thickness (100 m).




In [8]:
bedrock_thickness = 100 # m
K_df['b_sat_r'] = np.maximum((K_df['b_r_Shan'] - K_df['b_u']), 0)

K_df['K_s_wet_md'] = K_df['Ksat_Wet'] * 864
K_df['K_b_wet_md'] = (K_df['T'] - (K_df['K_s_wet_md'] * 8.35)) / bedrock_thickness

K_df['K_s_dry_md'] = K_df['Ksat_Dry'] * 864
K_df['K_b_dry_md'] = (K_df['T'] - (K_df['K_s_dry_md'] * 2.1)) / bedrock_thickness

K_df['K_s_dry_mod_md'] = np.where(K_df['b_sat_r'] > 2.1, 2.1 / K_df['b_sat_r'] * K_df['K_s_dry_md'], K_df['K_s_dry_md']) 
K_df['K_b_dry_mod_md'] = np.where(K_df['K_b_dry_md'] < 0.01, 0.01, K_df['K_b_dry_md'])

# K_df[K_df < 0.0] = np.nan

In [9]:
for column, item in K_df.iteritems():
    fname = '{}.tif'.format(column)
    data = item.values.reshape(natgrid.nrow, natgrid.ncol)
    natgrid.new_array = data
    dst = os.path.join('{}.tif'.format(column))
    natgrid.write_raster(dst, which_raster='new')

In [10]:
K_df.describe()

,T,Ksat_Wet,Ksat_Dry,b_r_Shan,b_r_Pell,b_u,b_sat_r,K_s_wet_md,K_b_wet_md,K_s_dry_md,K_b_dry_md,K_s_dry_mod_md,K_b_dry_mod_md
count,1.248848e+08,2.109327e+08,2.109327e+08,2.109327e+08,2.109327e+08,2.109327e+08,2.109327e+08,2.109327e+08,1.248848e+08,2.109327e+08,1.248848e+08,2.109327e+08,1.248848e+08
mean,5.568281e+02,5.160232e-02,1.021280e-02,1.980087e+01,7.871572e+01,1.178720e+01,1.337027e+01,4.458424e+01,-6.652745e-01,8.823873e+00,5.258522e+00,3.145010e+00,5.308929e+00
std,3.341011e+03,9.658267e-02,1.449629e-02,2.862894e+01,1.041350e+02,3.251646e+01,3.393406e+01,8.344743e+01,3.426560e+01,1.252480e+01,3.340676e+01,6.272447e+00,3.339730e+01
min,-7.926815e+04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-7.928852e+04,0.000000e+00,0.000000e+00,-8.069363e+02,0.000000e+00,-7.935679e+02,0.000000e+00,1.000000e-02
25%,8.981087e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-6.697996e+00,0.000000e+00,-3.036148e-02,0.000000e+00,1.000000e-02
50%,3.969384e+01,6.562218e-03,2.073885e-03,1.300943e+01,2.749318e+01,2.192134e-01,3.675770e+00,5.669756e+00,-1.847180e+00,1.791837e+00,1.134367e-01,4.370987e-01,1.134367e-01
75%,1.762876e+02,6.505843e-02,1.708171e-02,2.547066e+01,2.215856e+02,1.288906e+01,1.748574e+01,5.621048e+01,-2.300121e-01,1.475860e+01,1.288956e+00,3.137570e+00,1.288956e+00
max,1.509668e+05,8.249872e-01,9.652791e-02,1.057994e+03,2.550000e+02,2.405164e+02,7.929413e+04,7.127889e+02,1.507258e+03,8.340012e+01,1.509514e+03,8.340012e+01,1.509514e+03
